<a href="https://colab.research.google.com/github/owenc8/IAT360_final_project/blob/main/final_proj_CV_IAT360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

Making directories

In [ ]:
test_dir = '/content/drive/MyDrive/CV_final_project/Test'
train_dir = '/content/drive/MyDrive/CV_final_project/Train'

In [ ]:
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/images')
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/labels')
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/images/train')
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/images/val')
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/labels/train')
os.makedirs('/content/drive/MyDrive/CV_final_project/Final_data/labels/val')

In [ ]:
label_dir= "/content/drive/MyDrive/CV_final_project/Final_data/labels"
image_dir="/content/drive/MyDrive/CV_final_project/Final_data/images"

In [ ]:
#run it once for copying!
for file in os.listdir(train_dir):
    if file.endswith(".xml"):
        shutil.copy(os.path.join(train_dir, file), os.path.join(label_dir, "train"))
    if file.endswith(".jpg"):
        image = Image.open(os.path.join(train_dir, file))
        image = image.convert("RGB")
        new_filename = os.path.splitext(file)[0] + ".jpg"
        image.save(os.path.join(image_dir,"train", new_filename), "JPEG")
        #shutil.copy(os.path.join(train_dir, file), os.path.join(image_dir, "train"))


#Copy XML files and jpg files from the test folder to the folders created.
for file in os.listdir(test_dir):
    if file.endswith(".xml"):
        shutil.copy(os.path.join(test_dir, file), os.path.join(label_dir, "val"))
    if file.endswith(".jpg"):
        image = Image.open(os.path.join(test_dir, file))
        image = image.convert("RGB")
        new_filename = os.path.splitext(file)[0] + ".jpg"
        image.save(os.path.join(image_dir,"val", new_filename), "JPEG")
        #shutil.copy(os.path.join(test_dir, file), os.path.join(image_dir, "val"))